In [1]:
#import librairies 


from keras import backend as K
import tensorflow as tf
import pandas as pd 
import numpy as np
import math
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import collections
import progressbar
import time
from time import process_time
import sys
import os
import random
import gc

from sklearn import svm
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from imblearn.under_sampling import RandomUnderSampler 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import f1_score,classification_report,roc_auc_score,precision_score,recall_score, precision_recall_fscore_support 
from sklearn.model_selection import GridSearchCV,train_test_split
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout

#from sklearn_som.som import SOM
import networkx as nx
from networkx.algorithms import approximation

from GEM.gem.utils      import graph_util, plot_util
from GEM.gem.evaluation import visualize_embedding as viz
from GEM.gem.evaluation import evaluate_graph_reconstruction as gr
from GEM.gem.embedding.gf       import GraphFactorization
#from GEM.gem.embedding.sdne     import SDNE
#from argparse import ArgumentParser
#from GraphEmbedding.ge import DeepWalk
#from GraphEmbedding.ge import SDNE
from karateclub.graph_embedding import Graph2Vec
from karateclub.node_embedding.neighbourhood import HOPE
from karateclub.node_embedding.neighbourhood import DeepWalk


num_frame=200#arbitraire , a tester plus serieusement

pd.set_option('display.max_columns', 100)
print("import terminé a :",time.strftime("%H:%M:%S", time.localtime()))

import terminé a : 15:17:17


In [2]:
#ce dictionaire contien le nom de chaques colones avec la valeur du f1_score
#qu'ils on obtenu apres etre ajouté au data_set
#initialisé a 0
#-1,name1,name2,w,b

def convert_dist_to_zero_and_ones(data_set_dist, threshold=0.3):
    new_data_set = data_set_dist < threshold
    return new_data_set.astype(int)

def create_pair_dict(columns_list=[]):
    if (columns_list==[]):
        print("you forgot to input a list of columns")
        return []
    pair_dict=dict()
    for name1 in columns_list:
        for name2 in columns_list:# oui ce n'est pas optimisé
            pair_names=name1+"_"+name2
            reverse_pair_names=name2+"_"+name1
            if name1!=name2 and pair_names not in pair_dict.keys() and reverse_pair_names not in pair_dict.keys():
                pair_dict[pair_names]=[-1,name1,name2,0,0]
    
    return pair_dict

def create_monstruous_pair_data_set_deprecated(pair_dict,panda_data_set):
    columns_list = panda_data_set.columns.tolist()
    if (columns_list==[]):
        print("you forgot to input a data_set")
        return {},[]
    if (pair_dict=={}):
        print("you forgot to input a dict")
        return {},[]
    #pair_dict=dict()
    process_bar = progressbar.ProgressBar().start(max_value=len(pair_dict));i=0
    monstruous_pair_data_set=[]
    for pair in pair_dict.items():
        pair_dict[pair[0]][0]=i
        #print(pair[1][0],pair[1][1],pair[1][2])# value name1,name2
        tmp_data_set=panda_data_set[[pair[1][1],pair[1][2]]]
        tmp_w,tmp_b=linear_regresion(tmp_data_set)
        pair_dict[pair[0]][3]=tmp_w
        pair_dict[pair[0]][4]=tmp_b
        monstruous_pair_data_set.append(distances(tmp_data_set,tmp_w,tmp_b))
        process_bar.update(i);i+=1
    #the monstruous data_set should contain 0 and 1
    return pair_dict,monstruous_pair_data_set

def create_monstruous_pair_data_set(pair_dict,panda_data_set):
    columns_list = panda_data_set.columns.tolist()
    if (columns_list==[]):
        print("you forgot to input a data_set")
        return {},[]
    if (pair_dict=={}):
        print("you forgot to input a dict")
        return {},[]
    #pair_dict=dict()
    process_bar = progressbar.ProgressBar().start(max_value=len(pair_dict));i=0
    monstruous_pair_data_set=np.array([])
    for pair in pair_dict.items():
        #print(pair[1][0],pair[1][1],pair[1][2])# id,name1,name2,w,b
        pair_dict[pair[0]][0]=i
        tmp_data_set=panda_data_set[[pair[1][1],pair[1][2]]]
        tmp_w,tmp_b=linear_regresion(tmp_data_set)
        pair_dict[pair[0]][3]=tmp_w
        pair_dict[pair[0]][4]=tmp_b
        
        if(monstruous_pair_data_set.size ==0):
            monstruous_pair_data_set=distances_expand(tmp_data_set,tmp_w,tmp_b)
        else:
            monstruous_pair_data_set=np.concatenate((monstruous_pair_data_set,distances_expand(tmp_data_set,tmp_w,tmp_b)),axis=1)
        process_bar.update(i);i+=1
    #the monstruous data_set should contain 0 and 1
    return pair_dict,monstruous_pair_data_set

def make_one_graph(line ,pair_dict):
    g=nx.Graph()
    
    for pair in pair_dict.items():
        #print(pair[1][0],pair[1][1],pair[1][2])# id,name1,name2,w,b
        i= pair[1][0]
        arc_val=line[i]

        if arc_val ==1:
            name_1=pair[1][1]
            name_2=pair[1][2]
            g.add_edge(name_1,name_2)
    
    return g

def create_graphs(monstruous_binary_np_data_set,pair_dict):
    start = time.time()
    graph_list=[]
    process_bar = progressbar.ProgressBar().start(max_value=len(monstruous_binary_np_data_set));i=0
   
    for line in monstruous_binary_np_data_set:
        #print(type(line),line.shape)

        graph_list.append( make_one_graph(line ,pair_dict))
        process_bar.update(i);i+=1
    print("---graph construction = %s seconds ---" % (time.time() - start));start = time.time()
 
    return graph_list

def save_np(np_array,name):
    if(".npy" not in name):
        print("you save a numpy array in a file.npy")
        return 0
    np.save(name, np_array)

def load_np(file_name):
    return np.load(file_name)

    
############################################## liste des options d'extractions
def graph_max_degree(g):
    degrees = [val for (node, val) in g.degree()]
    maxd=max(degrees)
    return maxd
def geodesic_dist(graph):
    return nx.average_shortest_path_length(graph)

#special thanks to Francisco A. Rodrigues, University of São Paulo.
# http://conteudo.icmc.usp.br/pessoas/francisco
def degree_distribution(G):
    vk = dict(G.degree())
    vk = list(vk.values()) # we get only the degree values
    maxk = np.max(vk)
    mink = np.min(min)
    kvalues= np.arange(0,maxk+1) # possible values of k
    Pk = np.zeros(maxk+1) # P(k)
    for k in vk:
        Pk[k] = Pk[k] + 1
    Pk = Pk/sum(Pk) # the sum of the elements of P(k) must to be equal to one
    return kvalues,Pk
def shannon_entropy(G):
    k,Pk = degree_distribution(G)
    H = 0
    for p in Pk:
        if(p > 0):
            H = H - p*math.log(p, 2)
    return H

def contain_meso_scale(graph):
    
    return False    




def extract_info_from_one_graph(g, to_do_list):
    graph_property=np.array([])
    calc_result=0
    
    if("max_degree" in to_do_list):
        #print (" max degree")
        calc_result=graph_max_degree(g)
        graph_property=np.append(graph_property,calc_result)
    if("assortativity" in to_do_list):
        calc_result = nx.degree_assortativity_coefficient(g)
        graph_property=np.append(graph_property,calc_result)
    if("clustering" in to_do_list):
        calc_result= approximation.average_clustering(g, trials=1000, seed=10)
        graph_property=np.append(graph_property,calc_result)
    if("global_efficiency" in to_do_list):
        calc_result= nx.global_efficiency(g)
        graph_property=np.append(graph_property,calc_result)   
    if("geodesic_dist" in to_do_list):
        if nx.is_connected(g):
            calc_result=geodesic_dist(g)
        else:
            calc_result=len(g.nodes()) # = la dist la plus grande
        graph_property=np.append(graph_property,calc_result)
    if("contain_meso_scale" in to_do_list):#to do
        calc_result=contain_meso_scale(g)
        graph_property=np.append(graph_property,calc_result)
    if("Shannon_entropy" in to_do_list):#to do
        calc_result=shannon_entropy(g)
        graph_property=np.append(graph_property,calc_result)  
    #change this ()
    #            ()
    #            ()
    #      to this ()()()
    return np.expand_dims(graph_property,axis=0)

def extract_info_from_graphs(graph_list, to_do_list):
    
    monstruous_info_data_set=np.array([])
    process_bar = progressbar.ProgressBar().start(max_value=len(graph_list));i=0
   
    
    for graph in graph_list:
        
        if(monstruous_info_data_set.size ==0):
            monstruous_info_data_set=extract_info_from_one_graph(graph, to_do_list)
        else:
            monstruous_info_data_set=np.concatenate((monstruous_info_data_set,extract_info_from_one_graph(graph, to_do_list)),axis=0)
        process_bar.update(i);i+=1
    
    return monstruous_info_data_set



def create_graphs_to_delete(monstruous_binary_np_data_set,pair_dict):
    start = time.time()
    graph_list=[]
    process_bar = progressbar.ProgressBar().start(max_value=len(monstruous_binary_np_data_set));i=0
   
    for line in monstruous_binary_np_data_set:
        #print(type(line),line.shape)

        graph_list.append( make_one_graph(line ,pair_dict))
        process_bar.update(i);i+=1
    print("---graph construction = %s seconds ---" % (time.time() - start));start = time.time()
    return False


def extract_info_from_binary_data_set(monstruous_binary_np_data_set,pair_dict, to_do_list):
    start_time = process_time() 
    process_bar = progressbar.ProgressBar().start(max_value=len(monstruous_binary_np_data_set));i=0
    monstruous_info_data_set=np.array([])
    counter = 0
    contruct_time=0
    calcul_time=0
    for line in monstruous_binary_np_data_set:
        in_loop_time = process_time()
        graph=make_one_graph(line ,pair_dict)
        contruct_time = contruct_time+ process_time()  - in_loop_time
        
        in_loop_time = process_time()
        if(monstruous_info_data_set.size ==0):
            monstruous_info_data_set=extract_info_from_one_graph(graph, to_do_list)
        else:
            monstruous_info_data_set=np.concatenate((monstruous_info_data_set,extract_info_from_one_graph(graph, to_do_list)),axis=0)
        calcul_time = calcul_time+ process_time()  - in_loop_time
        process_bar.update(i);i+=1
        
        #####################################################################
        del graph # je delete le graph pour etre sur de liberer la memoire
        #gc.collect() #je n'appelle pas le garbageColector = cout 22 heures
        #calcul_time =  521.984375 contruct_time 59.3125
        #counter=counter+1
        #if(counter == 100000):  
            #print(process_time()  - start_time, "seconds");start_time = process_time()
            #print("calcul_time = ",calcul_time,"contruct_time",contruct_time)
            #return monstruous_info_data_set
            
        
    
    print("---graph extraction = %s seconds ---" % (process_time()  - start_time));start_time = process_time()
    return monstruous_info_data_set

# Function to find distance line equation ax + by + c =0
#y=mx+c  ----> mx -y +c =0  ---> b=-1 ,a=weight ,c=bias
def shortest_distance(x1, y1, a, b, c):
    
    d = abs((a * x1 + b * y1 + c)) / (math.sqrt(a * a + b * b))
    #print("Perpendicular distance is ",d)
    return d

def distances(data_set,w,b):
    np_data=data_set.to_numpy()
    np_dist=[]
    for line in np_data:
        x1=line[0]
        y1=line[1]
        np_dist.append(shortest_distance(x1, y1, w, -1, b))
    
    return np.array(np_dist)

def distances_expand(data_set,w,b):
    np_data=data_set.to_numpy()
    np_dist=[]
    for line in np_data:
        x1=line[0]
        y1=line[1]
        np_dist.append(shortest_distance(x1, y1, w, -1, b))
    
    return np.expand_dims( np.array(np_dist)  ,axis=1)
# y = mx +b
def line_equation_from_two_points(x1,y1,x2,y2):
    m=(y2-y1)/(x2-x1)
    b=y1-m*x1
    return m,b



def shortest_distance_test():
    print(shortest_distance(0, 0, 1, -1, 1))
    print(shortest_distance(-1, 0, 1, -1, 1))
    print(shortest_distance(1, 0, 1, -1, 1))
    print(math.sqrt(2))
    print(shortest_distance(3, 0, 1, -1, 1))
    print(math.sqrt(2*2+2*2))


def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

#https://www.machinelearningplus.com/deep-learning/linear-regression-tensorflow/
###############################################################################
################# partie linear regression ####################################
###############################################################################
###############################################################################
###############################################################################

def NormalizeData(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

def lin_reg_bootstrap(panda_data_set,nb_iter=1000):
    columns_list = panda_data_set.columns.tolist()
    x_train=NormalizeData(panda_data_set[columns_list[0]].to_numpy())
    y_train=NormalizeData(panda_data_set[columns_list[1]].to_numpy())
    data_set_size=len(x_train)
    m=0
    b=0
    counter=0
    for i in range(nb_iter):
        p1=random.randint(0, data_set_size-1)
        p2=random.randint(0, data_set_size-1)
        
        x1=x_train[p1]
        y1=y_train[p1]        
        x2=x_train[p2]
        y2=y_train[p2]
        if((x2-x1)!=0):
            m_tmp,b_tmp=line_equation_from_two_points(x1,y1,x2,y2)
            m=m+m_tmp
            b=b+b_tmp
        else:
            counter=counter+1
    m=m/(nb_iter-counter)
    b=b/(nb_iter-counter)
    return m,b
    
def linreg(x,weight,bias):#x is a np list
    y = weight*x + bias
    return y

# Define loss function (MSE)
def squared_error(y_pred, y_true):
    return tf.reduce_mean(tf.square(y_pred - y_true))

def not_squared_error(y_pred, y_true):
    return tf.reduce_mean(y_pred - y_true)

def linear_regresion(panda_data_set,boot_iter=1000):
    columns_list = panda_data_set.columns.tolist()
    #x_train=panda_data_set[columns_list[0]].to_numpy()
    #y_train=panda_data_set[columns_list[1]].to_numpy()
    x_train=NormalizeData(panda_data_set[columns_list[0]].to_numpy())
    y_train=NormalizeData(panda_data_set[columns_list[1]].to_numpy())
    #print("x_train max =",np.amax(x_train, axis=0))
    #print("y_train max =",np.amax(y_train, axis=0))
    learning_rate = 0.01
    # Number of loops for training through all your data to update the parameters
    training_epochs = 100
    
    boot_weight,boot_bias=lin_reg_bootstrap(panda_data_set,nb_iter=boot_iter)
    weight = tf.Variable(boot_weight)
    bias   = tf.Variable(boot_bias)
    #weight = tf.Variable(0.)
    #bias   = tf.Variable(0.)
    
    
    for epoch in range(training_epochs):

        # Compute loss within Gradient Tape context
        with tf.GradientTape() as tape:
            y_predicted = linreg(x_train,weight,bias)
            #loss = squared_error(y_predicted, y_train)
            loss = not_squared_error(y_predicted, y_train)
            # Get gradients
            gradients = tape.gradient(loss, [weight,bias])

            # Adjust weights
            weight.assign_sub(gradients[0]*learning_rate)
            bias.assign_sub(gradients[1]*learning_rate)

    
    
    return weight.numpy(),bias.numpy()

print("fonctions declaré a :",time.strftime("%H:%M:%S", time.localtime()))



fonctions declaré a : 15:17:17


In [3]:
program_start = time.time()

In [4]:
#%run "../Datatransformer.ipynb" import *
#%run "../Feature engineering.ipynb" import *

In [5]:
#--- import data_set = 20.337912797927856 seconds --- 20 -28 sec
start = time.time()

data_file= os.path.abspath('../../data')
data_folder= os.path.abspath('../../data')    # os.path ---> C:\Users\....
full_path=data_folder+'\\'+'fraudTrain.csv'   # unmodified train set
train_df=pd.read_csv(full_path)


full_path=data_file+'\\'+'fraudTest.csv'
test_df=pd.read_csv(full_path)



full_path=data_file+'\\'+'X_train_1_2_svm.csv'
xtrain_transformed_complique=pd.read_csv(full_path)
ytrain_transformed_complique=train_df['is_fraud'].iloc[:int(len(train_df)*0.8)]

full_path=data_file+'\\'+'X_val_1_2_svm.csv'
xval_transformed_complique=pd.read_csv(full_path)
yval_transformed_complique=train_df['is_fraud'].iloc[len(ytrain_transformed_complique):]

full_path=data_file+'\\'+'X_test_1_2_svm.csv'
xtest_transformed_complique=pd.read_csv(full_path)
ytest_transformed_complique=test_df['is_fraud']

#xtest_transformed_complique=pd.read_csv("../../data/X_test_1_2_svm.csv")
cols = xtrain_transformed_complique.columns.tolist()
print(cols)
print("--- import data_set = %s seconds ---" % (time.time() - start));start = time.time()


#X_train_1_2_favg, X_val_1_2_favg = dataTransformer(X_train, X_val=X_val, X_test=None, Y_train=None, config_name='config1_2')
#X_train_1_2_favg, X_test_1_2_favg = dataTransformer(X_train, X_test=X_test, X_val=None, Y_train=None, config_name='config1_2')


features_set_4 = ['amt', 'delta_avg_amt', 'avg_amt', 'delta_amt', 'avg_amt_category', 'delta_avg_amt_category', 'category', 'hour', 'delta_avg_amt_category_job', 'delta_amt_category', 'delta_time', 'gender', 'dob', 'avg_amt_state', 'delta_amt_merchant']
xtrain_transformed_simplifié=xtrain_transformed_complique[features_set_4]


['Unnamed: 0', 'merchant', 'category', 'amt', 'gender', 'state', 'zip', 'lat', 'long', 'city_pop', 'dob', 'unix_time', 'merch_lat', 'merch_long', 'delta_time', 'delta_amt', 'delta_time_category', 'delta_amt_category', 'delta_time_merchant', 'delta_amt_merchant', 'avg_amt', 'delta_avg_amt', 'avg_amt_category', 'delta_avg_amt_category', 'avg_amt_merchant', 'avg_amt_state', 'avg_amt_city', 'avg_amt_job', 'delta_avg_amt_category_job', 'month', 'day', 'hour']
--- import data_set = 21.138695001602173 seconds ---


In [6]:
dict_378 = create_pair_dict(cols)
print (len(dict_378))
print(math.sqrt(len(dict_378)))
print(len(cols))

496
22.271057451320086
32


In [7]:
#create data_set  ---create_monstruous_pair_data_set = 0.00195622444153 seconds ---
start = time.time()
print("--- create_graph_dist = %s seconds ---" % (time.time() - start));start = time.time()


--- create_graph_dist = 0.0 seconds ---


In [8]:

print("--- convert_dist_to_zero_and_ones = %s seconds ---" % (time.time() - start));start = time.time()


--- convert_dist_to_zero_and_ones = 0.014961004257202148 seconds ---


In [9]:
# load data

start = time.time()

to_do_list=["max_degree","assortativity","global_efficiency","geodesic_dist","Shannon_entropy"]
treshold_value=0.5
name="data/"+"train_graph_properties"+str(treshold_value)+".npy"
loaded_np=np.load(name)
df_addon = pd.DataFrame(loaded_np, columns = to_do_list)
X_train_plus_graph_df=pd.concat([xtrain_transformed_complique,df_addon])
X_train_plus_graph =np.concatenate((xtrain_transformed_complique,loaded_np),axis=1)


name="data/"+"val_graph_properties"+str(treshold_value)+".npy"
loaded_np=np.load(name)
df_addon = pd.DataFrame(loaded_np, columns = to_do_list)
X_val_plus_graph_df=pd.concat([xval_transformed_complique,df_addon])
X_val_plus_graph =np.concatenate((xval_transformed_complique,loaded_np),axis=1)

name="data/"+"test_graph_properties"+str(treshold_value)+".npy"
loaded_np=np.load(name)
df_addon = pd.DataFrame(loaded_np, columns = to_do_list)
X_test_plus_graph_df=pd.concat([xtest_transformed_complique,df_addon])
X_test_plus_graph =np.concatenate((xtest_transformed_complique,loaded_np),axis=1)


print("--- load data = %s seconds ---" % (time.time() - start));start = time.time()


--- load data = 0.8762392997741699 seconds ---


In [19]:
print(len(xtrain_transformed_complique),xtrain_transformed_complique.shape)

#xtrain_transformed_complique=xtrain_transformed_simplifié
xtrain_transformed_complique=xtrain_transformed_complique.drop(columns=['Unnamed: 0'])
xval_transformed_complique=xval_transformed_complique.drop(columns=['Unnamed: 0'])
xtest_transformed_complique=xtest_transformed_complique.drop(columns=['Unnamed: 0'])


print(len(xtrain_transformed_complique),xtrain_transformed_complique.shape)


1037340 (1037340, 31)
1037340 (1037340, 31)


In [11]:
# max_iter=1000
#--- svm fit = 185.69944405555725 seconds ---
#---svm y_train_pred = 348.6557650566101 seconds ---
#---svm y_test_pred = 184.23344445228577 seconds ---
# classification report = trash = f1-score 0.02

# max_iter=3333  sous entrainement
# max_iter=6666  sur entrainement
# max_iter=10000  time = 2 h
print("--------------------------------------------------------")
print("data set is xtrain_transformed_simplifié")
start = time.time()
svm_model = svm.SVC(kernel="rbf", gamma = 0.02, C=10)#, max_iter=100000)
svm_model.fit(xtrain_transformed_simplifié,ytrain_transformed_complique)
print("--- svm fit = %s seconds ---" % (time.time() - start));start = time.time()
y_train_pred=svm_model.predict(xtrain_transformed_simplifié)
print("---svm y_train_pred = %s seconds ---" % (time.time() - start));start = time.time()
#y_test_pred=svm_model.predict(xtest_transformed_complique)
print("---SVM y_test_pred = %s seconds ---" % (time.time() - start));start = time.time()
print(classification_report(ytrain_transformed_complique,y_train_pred))
#print(classification_report(ytest_transformed_complique,y_test_pred))
#cl=classification_report(ytest_transformed_complique,Y_test_pred,output_dict=True)
print("--------------------------------------------------------")

--- svm fit = 1271.3207511901855 seconds ---
---svm y_train_pred = 637.7386474609375 seconds ---
---SVM y_test_pred = 0.0 seconds ---
              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1031372
           1       0.97      0.76      0.85      5968

    accuracy                           1.00   1037340
   macro avg       0.98      0.88      0.93   1037340
weighted avg       1.00      1.00      1.00   1037340

--------------------------------------------------------


In [12]:
# max_iter=1000
#--- svm fit = 185.69944405555725 seconds ---
#---svm y_train_pred = 348.6557650566101 seconds ---
#---svm y_test_pred = 184.23344445228577 seconds ---
# classification report = trash = f1-score 0.02

# max_iter=3333  sous entrainement
# max_iter=6666  sur entrainement
# max_iter=10000  time = 2 h
print("--------------------------------------------------------")
print("data set is xtrain_transformed_complique")
start = time.time()
svm_model = svm.SVC(kernel="rbf", gamma = 0.02, C=10)#, max_iter=100000)
svm_model.fit(xtrain_transformed_complique,ytrain_transformed_complique)
print("--- svm fit = %s seconds ---" % (time.time() - start));start = time.time()
y_train_pred=svm_model.predict(xtrain_transformed_complique)
print("---svm y_train_pred = %s seconds ---" % (time.time() - start));start = time.time()
#y_test_pred=svm_model.predict(xtest_transformed_complique)
print("---SVM y_test_pred = %s seconds ---" % (time.time() - start));start = time.time()
print(classification_report(ytrain_transformed_complique,y_train_pred))
#print(classification_report(ytest_transformed_complique,y_test_pred))
#cl=classification_report(ytest_transformed_complique,Y_test_pred,output_dict=True)
print("--------------------------------------------------------")

--- svm fit = 2424.898648738861 seconds ---
---svm y_train_pred = 1030.2677960395813 seconds ---
---SVM y_test_pred = 0.0 seconds ---
              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1031372
           1       0.99      0.84      0.91      5968

    accuracy                           1.00   1037340
   macro avg       1.00      0.92      0.95   1037340
weighted avg       1.00      1.00      1.00   1037340

--------------------------------------------------------


In [20]:
y_test_pred=svm_model.predict(xtest_transformed_complique)
print(classification_report(ytest_transformed_complique,y_test_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    553574
           1       0.91      0.66      0.76      2145

    accuracy                           1.00    555719
   macro avg       0.95      0.83      0.88    555719
weighted avg       1.00      1.00      1.00    555719



In [13]:
# WHY DO YOU NOT WORK ???
# what is it that makes you slow ? when its literaly the same code ?
# je rajoute max_iter on vera qui est le plus fort

In [14]:
# svm O( n² x nb_features)
not_too_long=False
if(not_too_long):#72h d'execution 
    
    start = time.time()
    svm_model = svm.SVC(kernel="rbf", gamma = 0.02, C=10)
    svm_model.fit(xtrain_transformed_complique,ytrain_transformed_complique)
    y_train_pred=svm_model.predict(xtrain_transformed_complique)
    y_test_pred=svm_model.predict(xtest_transformed_complique)
    print("---SVM sans modification = %s seconds ---" % (time.time() - start));start = time.time()
    print(classification_report(ytrain_transformed_complique,y_train_pred))
    print(classification_report(ytest_transformed_complique,y_test_pred))
    cl=classification_report(ytest_transformed_complique,y_test_pred,output_dict=True)
    print("--------------------------------------------------------")
    ####
    svm_model.fit(X_train_plus_graph,ytrain_transformed_complique)
    y_train_pred_graph=svm_model.predict(X_train_plus_graph)
    y_test_pred_graph=svm_model.predict(X_test_plus_graph)
    print("---SVM avec plus d informations = %s seconds ---" % (time.time() - start));start = time.time()
    print(classification_report(ytrain_transformed_complique,y_train_pred_graph))
    print(classification_report(ytest_transformed_complique,y_test_pred_graph))
    cl2=classification_report(ytest_transformed_complique,y_test_pred_graph,output_dict=True)
    print("recall diff =",cl["1"]["recall"]-cl2["1"]["recall"])
    print("f1-score diff =",cl["1"]["f1-score"]-cl["1"]["f1-score"])
    print("--------------------------------------------------------")

In [15]:
#WARNING:tensorflow:Early stopping conditioned on metric `Recall` which is not available. 
#Available metrics are: loss,f1,recall,val_loss,val_f1,val_recall

In [16]:
################### big exec ###########################################
########################################################################
########################################################################
########################################################################
########################################################################
########################################################################
########################################################################
########################################################################
########################################################################
########################################################################
########################################################################
########################################################################
